In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as st
import time
import datetime as dt
import sys as sys
##import dask.dataframe as dd



In [ ]:
#from dask.distributed import Client, LocalCluster
#cluster = LocalCluster() 
#cluster
#client = Client(cluster)

In [ ]:
import ipyparallel
cluster = ipyparallel.Client(profile='parallel-profile',sshserver='paperspace@172.83.14.24',password='Feb)@2018')
#cluster = ipyparallel.Client(profile='parallel-profile',sshserver='paperspace@172.83.14.24',password='Feb)@2018')
print('profile:', cluster.profile)
print("IDs:", cluster.ids) # Print process id numbers
print(cluster)

In [ ]:
LESSON_DATA_FOLDER = './data/'

#from datetime import datetime as dt

def index_array(length):
    result = [x for x in range(length)]   
    return result

def parse_date(date):
    if str(date) == '' or date == None:
        return None
    else:
        return dt.strptime(date,'%Y-%m-%d')    

def parse_int(i, defaultValue=None):    
   # print (defaultValue)
    if ( i == None or str(i) == '' or str(i) == 'NaN' or i == np.NaN) :
        return defaultValue   
    else:
        try:
            #print ('convert', int(i))
            return int(float(i))
        except:
            return i
    
def parse_float(f):
    if str(f) == '' or f == None:
        return None
    else:
        try:
            return float(f)
        except:
            return f
    
def parse_bool(boolean):
    if str(boolean) == '' or boolean == None:
        return None
    else:        
        return boolean =='True' 
    
def split(data, delimiter=',' ):
    #print ('data: ', data)    
    if(data == '' or data == None or str(data) == None):
        return np.array([None])
    else:
        #return np.array(str(data).lower().split(delimiter))    
        lst = np.array(str(data).split(delimiter))       
        return lst    
    
def lower(data):
    if(data == '' or data == None):
        return None
    else:
        return str(data).lower() 
    
def upper(data):
    if(data == '' or data == None):
        return None
    else:
        return str(data).upper()     

def replaceNaN(data):
    if(np.isnan(data)): 
        return None
    else:
        return data
    

# Python does not have switch statment, rather use dict approach
parser = {
        'int':parse_int,
        'date':parse_date,
        'bool':parse_bool
    }


fileColumnMapping = {
'title.basics.tsv': {
      'index_col': None, 
      'dtype' : {'tconst':np.dtype('S'),
                 'titleType':np.dtype('S'), 
                 'primaryTitle':np.dtype('S'),
                 'originalTitle':np.dtype('S'),
                 'isAdult':np.dtype('S'),
                 'startYear':np.dtype('S'),
                 'endYear':np.dtype('S'),             
                 'runtimeMinutes':np.dtype('S'),
                 'genres':np.dtype('S')
                    },      
      'filePath':LESSON_DATA_FOLDER + 'title.basics.tsv',
      'to_replace':{
          'titleType':{'\\N':None},
          'primaryTitle':{'\\N':None},
          'originalTitle':{'\\N':None},
          'startYear':{'\\N':None},
          'endYear':{'\\N':None},
          'runtimeMinutes':{'\\N':None}
      },
     'true_values':['1'],
     'false_values':['0'],     
     'usecols':['tconst','titleType','primaryTitle','originalTitle','isAdult',
                'startYear','endYear','runtimeMinutes','genres'],
     'converters' : {
                 'primaryTitle':[{'function':lower, 'args':None}],
                 'titleType':[{'function':lower, 'args':None}],
                 'originalTitle':[{'function':lower, 'args':None}],
                 'isAdult':[{'function':parse_bool, 'args':None}],
                 'startYear':[{'function':parse_int, 'args':None}] ,
                 'endYear':[{'function':parse_int, 'args':None}]  ,
                 'runtimeMinutes':[{'function':parse_int, 'args':None}],
                 'genres': [{'function':split, 'args':(',',)}]
                 
                }
    },
'title.crew.tsv': {
      'index_col': None, 
      'dtype' : {'tconst':np.dtype('S'),'directors':np.dtype('S') ,'writers':np.dtype('S')  },
      'split' : ['directors','writers'],
      'filePath':LESSON_DATA_FOLDER + 'title.crew.tsv',
      'to_replace':{
          'directors':{'\\N':None},
          'writers':{'\\N':None},          
      },
     'true_values':None,
     'false_values':None,     
     'usecols': None,
     'converters' : {                 
                 'writers':[{'function':split, 'args':(',',)}],
                 'directors': [{'function':split, 'args':(',',)}]
                }
    }, 
'title.episode.tsv': {
      'index_col': None, 
      'dtype' : {'tconst':np.dtype('S'),
                 'parentTconst':np.dtype('S'),
                 'seasonNumber':np.dtype('S'),  
                 'episodeNumber':np.dtype('S')  
                },
      'split' :None,
      'filePath':LESSON_DATA_FOLDER + 'title.episode.tsv',
      'to_replace':{
          'seasonNumber':{'\\N':None},
          'episodeNumber':{'\\N':None},                   
      },
      'true_values':None,
      'false_values':None,      
      'usecols': None,
      'converters' : {                 
                 'seasonNumber':[{'function':parse_int, 'args':None}],
                 'episodeNumber': [{'function':parse_int, 'args':None}]
                }
    },
'title.principals.tsv': {
      'index_col': None, 
      'dtype' : {'tconst':np.dtype('S'),
                 'ordering':np.dtype('S'),
                 'nconst':np.dtype('S'),
                 'category':np.dtype('S'),
                 'job':np.dtype('S'),                 
                 'characters':np.dtype('S'),                 
                },
      'split' : None,
      'filePath':LESSON_DATA_FOLDER + 'title.principals.tsv',
      'to_replace':{
          'job':{'\\N':None},
          'characters':{'\\N':None},                   
      },
      'true_values':None,
      'false_values':None,
      'converters' : {                 
                 'ordering':[{'function':parse_int, 'args':None}]             
                },
      'usecols': None      
    },    
'title.ratings.tsv': {
      'index_col': None, 
      'dtype' : {'tconst':np.dtype('S'),'averageRating':np.float64 ,'numVotes':np.int32  },
      'split' :None,
      'filePath':LESSON_DATA_FOLDER + 'title.ratings.tsv',
      'to_replace':None,
      'true_values':None,
      'false_values':None,      
      'usecols': None,
      'converters' : {                 
                 'numVotes':[{'function':parse_int, 'args':None}]               
                }    
    },  
'name.basics.tsv': {
      'index_col': None, 
      'dtype' : {'nconst':np.dtype('S'),
                 'primaryName':np.dtype('S') ,
                 'birthYear':np.dtype('S')  ,
                 'deathYear':np.dtype('S'),
                 'primaryProfession':np.dtype('S'),
                 'knownForTitles':np.dtype('S')
                },      
      'filePath':LESSON_DATA_FOLDER + 'name.basics.tsv',
      'to_replace':{
          'primaryProfession':{'\\N':None},
          'knownForTitles':{'\\N':None},    
          'birthYear':{'\\N':None},    
          'deathYear':{'\\N':None},       
      },
      'true_values':None,
      'false_values':None,
      'usecols': None,    
      'converters' : {                 
             'primaryName':[{'function':lower, 'args':None}],                 
             'birthYear':[{'function':parse_int, 'args':(0,)}],                 
             'deathYear':[{'function':parse_int, 'args':(0,)}],
             'primaryProfession' :[{'function':split, 'args':(',',)}],
             'knownForTitles':[{'function':split, 'args':(',',)}],          
            } 
    },    
}

In [ ]:
import pandas as pd

def getMapping(file):
    return fileColumnMapping.get(file)

def callFunction(columnData, **funDict):
    
    converFuns = funDict[columnData.name] #[{'function':split, 'args':(',')}]
    
    if(converFuns != None):
        for funSpec in converFuns:
            params = funSpec['args']
            #print ('args: ', params, ' <> ', params == None)
            if(params == None):                
                columnData = columnData.apply(funSpec['function'] )
            else:            
                columnData = columnData.apply(funSpec['function'], args=params )
            
    return columnData

def readFile(file, nrows=None ):   
    print ('Start: ' + time.strftime("%Y-%m-%d %H:%M"))
    dview = cluster[:]
    mapping = getMapping(file)
    dtype = mapping['dtype']
    usecols = list(dtype.keys())
    
    dview.scatter(
        "df", 
        pd.read_table(mapping['filePath'], 
                       index_col=mapping['index_col'], 
                       dtype = dtype, 
                       #na_values = ['//N'],
                       true_values= mapping['true_values'],
                       false_values= mapping['false_values'],                       
                       usecols=usecols,
                       nrows =nrows 
                      )
    )
    df = pd.concat([i for i in dview["df"]])
    
    df.fillna(method='pad', inplace=True)
    if(mapping['to_replace']!= None):
        df.replace(to_replace=mapping['to_replace'],method='pad', inplace=True)
    
    converters = mapping['converters']
    if(converters!= None):
        cols = list(converters.keys())
        df[cols] = df[cols].apply(callFunction, **converters)
        
    print ('End: ' + time.strftime("%Y-%m-%d %H:%M"))
    return df

In [9]:

names = readFile('name.basics.tsv')#.compute()
print (time.strftime("%Y-%m-%d %H:%M"))
'''
2018-03-01 14:56
2018-03-01 14:58
'''


2018-03-01 14:56
2018-03-01 14:58


In [ ]:
crew = readFile('title.crew.tsv')#.compute()
crew.info()

In [10]:
names.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8445648 entries, 0 to 8445647
Data columns (total 6 columns):
nconst               object
primaryName          object
birthYear            int64
deathYear            int64
primaryProfession    object
knownForTitles       object
dtypes: int64(2), object(4)
memory usage: 386.6+ MB


In [ ]:
#print (time.strftime("%Y-%m-%d %H:%M"))